In [13]:
# Homework 7
# Jimena Stephenson

In [31]:
# import modules
import numpy as np
import astropy.units as u
import astropy.constants as const
from CenterofMass import CenterOfMass
#from OrbitCOM import          
# from astropy.constants import G
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Latex
%matplotlib inline


In [85]:

#################### M33 Analytic Obrbit ##########################

class M33AnalyticOrbit:
    # Class to define a series of functions that will determine the acceleration
    # M33 feels from M31 and integrate its current position and velocity forward in time
    
    def __init__(self, filename):
        # initialize the class
        # input:
        #      file where the integrated orbit will be stored
        self.filename = filename
         
        self.G = const.G.to(u.kpc**3/u.Msun/u.Gyr**2).value
        
        #initialize the relative position and velocity of M33 to M31
        M31COM = CenterOfMass("M31_000.txt", 2)
        M33COM = CenterOfMass("M33_000.txt", 2)
        
        
        # M31:   store the position and velocity COM                                                                           
        M31_COM_P = M31COM.COM_P(0.1).value
        M31_COM_V = M31COM.COM_V(M31_COM_P[0]*u.kpc,M31_COM_P[1]*u.kpc,M31_COM_P[2]*u.kpc).value


        # M33:   store the position and velocity COM                                                                           
        M33_COM_P = M33COM.COM_P(0.1).value
        M33_COM_V = M33COM.COM_V(M33_COM_P[0]*u.kpc,M33_COM_P[1]*u.kpc,M33_COM_P[2]*u.kpc).value
        
        # Determine the relative position between M33 and M31
        self.x = M33_COM_P[0] - M31_COM_P[0]
        self.y = M33_COM_P[1] - M31_COM_P[1]
        self.z = M33_COM_P[2] - M31_COM_P[2]
        self.r0 = np.sqrt(self.x**2 + self.y**2 + self.z**2)
       

        # Determine the relative velocity between M33 and M31 
        self.vx = M33_COM_V[0] - M31_COM_V[0]
        self.vy = M33_COM_V[1] - M31_COM_V[1]
        self.vz = M33_COM_V[2] - M31_COM_V[2]
        self.v0 = np.sqrt(self.vx**2 + self.vy**2 + self.vz**2)
        
        
        # Store the scale lengths and masses for each compponen in M31
        self.rdisk = 5 #kpc
        self.Mdisk = 0.12 * 1e12 #Msun
        self.rbulge = 1  
        self.Mbulge = 0.019 * 1e12
        self.rhalo = 61  
        self.Mhalo = 1.921 * 1e12
        
        
        
    ############## Define acceleration Terms #################
        
    # Define functions that will compute the gravitaninal acceleration vectors dfrom 
    # 3 components of the M31 galaxy: Halo, Bulge & disk 
    # The gravitationall acceleration induced by a Hernquist profile is given by:
    # a = -(G*M)/(r(Ra + r))  r direction
    # where M is the total halo or bulge mass and Ra is the corresponging scale length
        
    def HernquistAccel(self, M, r_a, x, y, z):
        # Function that comptues the gravitanional acceleration induced by a Hernquist profile
        # inputs:
        #       M= total halo or bulge mass, r_a = corresponding scale length, x, y, z
        # returns:
        #        the acceleration vector from a Hernquist potential 
        r = np.array([x,y,z])
            
        rmag = np.sqrt(x^2 + y^2 + z^2)
            
        return -(self.G*M) * r /(rmag*(r_a + rmag**2)) 
        
    def MiyamotoNagaiAccel(self, Md, rd, x, y, z):
        # This function is an aproximation that mimics the exponential disk profile at 
        # distances far from the disk. It is called a Miyamoto_Nagai 1975 profile.
        # phi_mn(r) = -G*M_disk / sqrt(R^2 + B^2)
        # where R = sqrt(x^2 + y ^2) and B = rd + sqrt(z^2 + zd^2)
        # where zd = self.rdisk/5.0, rd = self.rdisk
        # inputs:
        #       M= Mdisk, rd= self.rdisk, x, y, z
        # returns:
        #       acceleration vector from a Miyamoto-Nagai profile
            
        R = np.sqrt(x**2 + y**2)
            
        zd = rd /5.0
              
        B = rd + np.sqrt(z**2 + zd**2)
            
        ax = - (self.G*Md)*x/(R**2 + B**2)**1.5
            
        ay = - (self.G*Md)*y/(R**2 + B**2)**1.5
            
        az = - (self.G*Md)*B*z/((R**2 + B**2)**1.5 * np.sqrt(z**2 + zd**2))
            
        a = np.array([ax,ay,az])
            
        return a
        
    def M31Accel(self, r):
        # Function that sums up all acceleration vectors from each galaxy component
        # input: 3d position vector (x,y,z)
        # retrn a 3d vector of the total acceleration
        # aH = acceleration vector from halo
        # aB = acceleration vector from bulge
        # aD = acceleration vector from disk
            
        aH = self.HernquistAccel(self.Mhalo, self.rhalo, r[0], r[1], r[2])
        aB = self.HernquistAccel(self.Mbulge, self.rbulge, r[0], r[1], r[2])
        aD = self.MiyamotoNagaiAccel(self.Mdisk,self.rdisk,r[0],r[1],r[2])
            
        atot = aH + aB + aD
            
        return atot
        
    ############# Build an integrator ################
        
    # We nned to integrate the equations of motion in dirder to solve for M33 orbit
    # r double dot = - DEL(phi)tot = - DEL[Phi_halo(r) + Phi_bulge(r) + Phi_disk(r)]
    # r double dot = a_halo + a_bulge + a_disk
    # where phi is the potential for each galaxy component
    # We need to treat M33 as a point mass and do a "leap frog" given that a is a pure 
    # function of r 
        
    def LeapFrog(self, dt, r, v):
        # This function integrates the aquations of motion forward in time
        # inputs:
        #       dt = time interval for integration
        #       r = a starting position VECTOR for the M33 COM position relative to M31
        #       v = a starting velocity VECTOR for the M33 relative to M31
        # Need to update the postions and velocities using standard kinematic equations 
            
        # 3D position vector of M33 COM updated half timestep dt using the current velocity and position vector
        # r(n+1/2) = r + v * (dt/2)
        rhalf = r + v *(dt/2)
            
        # COM postion and velocity vectors updated a full time step using acceleration at the 1/2 
        # timestep
        # v(n+1) = v + a(n+1/2)dt  
        # r(n+1) = r(n+1/2) + v(n+1)*dt/2
        vn1 = v + self.M31Accel(rhalf) * dt
        rn1 = rhalf + vn1 * (dt/2)
            
        return vn1, rn1
        
        
    #################### Integrate Orbit #############################
        
    def OrbitIntegrator(self, t0, dt, tmax):
        # This function loops over the LeapFrog integrator to solve the equations of motion 
        # and computes the future orbit of M33 for 10 Gyr into the future
            
        # initial conditions

        t = t0
            
        orbit = np.zeros([int(tmax/dt)+2, 7]) # empty array
            
        orbit[0] = t0, *tuple(self.r0), *tuple(self.v0) # first row of the orbit
            
        i = 1
            
        while t <= tmax:
                
            t = t +dt         # upadting time            
            orbit[i, 0] = t   #  store the time in the first elemnt of the ith row 
            r, v = self.LeapFrog(dt, orbit[(i-1), 1:4], orbit[(i-1), 4:]) #advance the position and velocity
            orbit[i,1:4] = r   # store the new position vector  in column 1,2,3
            orbit[i,4] = v   # store the new position vector  in column 4,5,6
            i += 1           # update counter
                
           
        # save the output file.        
        np.savetxt(self.filename,Orbit,fmt = "%11.3f"*7, comments='#',
                header="{:>10s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}"\
                        .format('t', 'x', 'y', 'z', 'vx', 'vy', 'vz'))
            
            
            
        
            
            
            
        
        
        
    
    
        

In [86]:
M33O = M33AnalyticOrbit('orbit.txt')

In [87]:
M33O.OrbitIntegrator(0, 0.5, 10)


TypeError: 'numpy.float64' object is not iterable